<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_3_part_3_praktica/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%B0_%D1%80%D1%96%D1%88%D0%B5%D0%BD%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from sklearn.linear_model import Ridge

In [20]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [21]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [22]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Очистіть дані від викидів(лише `price`), розділіть на тренувальну та тестову чатини

In [23]:
df.describe()

,kms_driven,engine(cc),price(in lakhs)
count,1553.000000,1.550000e+03,1553.000000
mean,52841.931101,1.471857e+10,166.141494
std,40067.800347,2.185629e+11,3478.855090
min,620.000000,5.000000e+00,1.000000
25%,30000.000000,1.197000e+03,4.660000
50%,49134.000000,1.462000e+03,7.140000
75%,70000.000000,1.995000e+03,17.000000
max,810000.000000,3.258640e+12,95000.000000


In [24]:
Q1 = df['price(in lakhs)'].quantile(0.25)
Q3 = df['price(in lakhs)'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['price(in lakhs)'] >= (Q1 - 1.5 * IQR)) & (df['price(in lakhs)'] <= (Q3 + 1.5 * IQR))]

In [25]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [26]:
train_df.head()
test_df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
53,Diesel,64000,1995.0,25.50
1279,Petrol,36015,1197.0,3.61
1501,CNG,49999,1373.0,6.34
610,Petrol,14000,1462.0,19.90
1434,Petrol,60467,998.0,2.80


# Завдання 2
Створіть Pipeline для обробки даних

In [27]:
numeric_features = ['kms_driven', 'engine(cc)']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])
categorical_features = ['fuel_type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print(preprocessor)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['kms_driven', 'engine(cc)']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['fuel_type'])])


# Завдання 3
Створіть остаточну модель та натренуйте її

In [28]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

X_train = train_df.drop('price(in lakhs)', axis=1)
y_train = train_df['price(in lakhs)']
X_test = test_df.drop('price(in lakhs)', axis=1)
y_test = test_df['price(in lakhs)']



In [29]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['kms_driven', 'engine(cc)']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['fuel_type'])])),
                ('regressor', LinearRegression())])

# Завдання 4
Виведіть основні метрики моделі

In [30]:
y_pred = model.predict(X_test)

In [31]:
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"R^2 Score: {r2_score(y_test, y_pred)}")

Mean Squared Error: 42.652244297545934
R^2 Score: 0.17265127389904278


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

In [32]:
ridge_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', Ridge(alpha=1.0))])

In [33]:
ridge_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.57825e-26): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['kms_driven', 'engine(cc)']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['fuel_type'])])),
                ('regressor', Ridge())])

In [34]:
y_pred_ridge = ridge_model.predict(X_test)

In [35]:
print(f"Ridge Mean Squared Error: {mean_squared_error(y_test, y_pred_ridge)}")
print(f"Ridge R^2 Score: {r2_score(y_test, y_pred_ridge)}")

Ridge Mean Squared Error: 42.684132416096936
Ridge R^2 Score: 0.17203272276075143


# Завдання 6
Збережіть модель

In [36]:
joblib.dump(ridge_model, 'used_car_price_ridge_model.pkl')

['used_car_price_ridge_model.pkl']

In [39]:
loaded_model = joblib.load('used_car_price_ridge_model.pkl')

In [37]:
data = pd.DataFrame({'fuel_type': ['Diesel'],
                     'kms_driven': [48000],
                     'engine(cc)': [1832]})

In [40]:
predicted_price = loaded_model.predict(data)
print(f"Прогнозована ціна: {predicted_price[0]} лакхів")

Прогнозована ціна: 13.918445359096367 лакхів
